# Matrix Factorization

In [1]:
import pandas as pd 
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from scipy import sparse 

class MF(object):
    """docstring for CF"""
    def __init__(self, Y_data, K, lam = 0.1, Xinit = None, Winit = None, 
                 learning_rate = 0.5, max_iter = 1000, print_every = 100, user_based = 1):
        self.Y_raw_data = Y_data
        self.K = K
        self.lam = lam
        self.learning_rate = learning_rate
        self.max_iter = max_iter
        self.print_every = print_every
        self.user_based = user_based
        # number of users and items. Remember to add 1 since id starts from 0
        self.n_users = int(np.max(Y_data[:, 0])) + 1 
        self.n_items = int(np.max(Y_data[:, 1])) + 1
        
        if Xinit is None: 
            self.X = np.random.randn(self.n_items, K)
        else:
            self.X = Xinit 
        
        if Winit is None: 
            self.W = np.random.randn(K, self.n_users)
        else: 
            self.W = Winit
            
        #self.all_users = self.Y_data[:,0] # all users (may be duplicated)
        self.n_ratings = Y_data.shape[0]
        # normalized data
        self.Y_data_n = self.Y_raw_data.copy()

    def normalize_Y(self):
        if self.user_based:
            user_col = 0
            item_col = 1
            n_objects = self.n_users
        else:
            user_col = 1
            item_col = 0 
            n_objects = self.n_items

        users = self.Y_raw_data[:, user_col] 
        self.mu = np.zeros((n_objects,))
        for n in range(n_objects):
            # row indices of rating done by user n
            # since indices need to be integers, we need to convert
            ids = np.where(users == n)[0].astype(np.int32)
            # indices of all ratings associated with user n
            item_ids = self.Y_data_n[ids, item_col] 
            # and the corresponding ratings 
            ratings = self.Y_data_n[ids, 2]
            # take mean
            m = np.mean(ratings) 
            if np.isnan(m):
                m = 0 # to avoid empty array and nan value
            self.mu[n] = m
            # normalize
            self.Y_data_n[ids, 2] = ratings - self.mu[n]
            
    
    def loss(self):
        L = 0 
        for i in range(self.Y_data_n.shape[0]):
            # user, item, rating
            n, m, rate = int(self.Y_data_n[i, 0]), int(self.Y_data_n[i, 1]), self.Y_data_n[i, 2]
            L += 0.5*(rate - self.X[m, :].dot(self.W[:, n]))**2
            
        # regularization, don't ever forget this 
        L /= self.n_ratings
        L += 0.5*self.lam*(np.linalg.norm(self.X, 'fro') + np.linalg.norm(self.W, 'fro'))
        return L 

    
    def get_items_rated_by_user(self, user_id):
        """
        get all items which are rated by user n, and the corresponding ratings
        """
        # y = self.Y_data_n[:,0] # all users (may be duplicated)
        # item indices rated by user_id
        # we need to +1 to user_id since in the rate_matrix, id starts from 1 
        # while index in python starts from 0
        ids = np.where(self.Y_data_n[:,0] == user_id)[0] 
        item_ids = self.Y_data_n[ids, 1].astype(np.int32) # index starts from 0 
        ratings = self.Y_data_n[ids, 2]
        return (item_ids, ratings)
        
        
    def get_users_who_rate_item(self, item_id):
        """
        get all users who rated item m and get the corresponding ratings
        """
        ids = np.where(self.Y_data_n[:,1] == item_id)[0] 
        user_ids = self.Y_data_n[ids, 0].astype(np.int32)
        ratings = self.Y_data_n[ids, 2]
        return (user_ids, ratings)
        
    def updateX(self):
        for m in range(self.n_items):
            user_ids, ratings = self.get_users_who_rate_item(m)
            Wm = self.W[:, user_ids]
            grad_xm = -(ratings - self.X[m, :].dot(Wm)).dot(Wm.T)/self.n_ratings + \
                                               self.lam*self.X[m, :]
            self.X[m, :] -= self.learning_rate*grad_xm.reshape((self.K,))
    
    def updateW(self):
        for n in range(self.n_users):
            item_ids, ratings = self.get_items_rated_by_user(n)
            Xn = self.X[item_ids, :]
            grad_wn = -Xn.T.dot(ratings - Xn.dot(self.W[:, n]))/self.n_ratings + \
                        self.lam*self.W[:, n]
            self.W[:, n] -= self.learning_rate*grad_wn.reshape((self.K,))
    
    def fit(self):
        self.normalize_Y()
        for it in range(self.max_iter):
            self.updateX()
            self.updateW()
            if (it + 1) % self.print_every == 0:
                rmse_train = self.evaluate_RMSE(self.Y_raw_data)
                print('iter =', it + 1, ', loss =', self.loss(), ', RMSE train =', rmse_train)
    
    
    def pred(self, u, i):
        """ 
        predict the rating of user u for item i 
        if you need the un
        """
        u = int(u)
        i = int(i)
        
        if self.user_based:
            bias = self.mu[u]
        else: 
            bias = self.mu[i]
        pred = self.X[i, :].dot(self.W[:, u]) + bias 
        if pred < 1:
            return 1 
        if pred > 5: 
            return 5 
        return pred 
        
    
    def pred_for_user(self, user_id):
        ids = np.where(self.Y_data_n[:, 0] == user_id)[0]
        items_rated_by_u = self.Y_data_n[ids, 1].tolist()              
        
        y_pred = self.X.dot(self.W[:, user_id]) + self.mu[user_id]
        predicted_ratings= []
        for i in range(self.n_items):
            if i not in items_rated_by_u:
                predicted_ratings.append((i, y_pred[i]))
        
        return predicted_ratings
    
    def evaluate_RMSE(self, rate_test):
        n_tests = rate_test.shape[0]
        SE = 0 # squared error
        for n in range(n_tests):
            pred = self.pred(rate_test[n, 0], rate_test[n, 1])
            SE += (pred - rate_test[n, 2])**2 

        RMSE = np.sqrt(SE/n_tests)
        return RMSE
        
r_cols = ['user_id', 'item_id', 'rating']
ratings = pd.read_csv('ex.dat', sep = ' ', names = r_cols, encoding='latin-1')
Y_data = ratings.as_matrix()


rs = MF(Y_data, K = 2, max_iter = 1000, print_every = 1000)

rs.fit()
rs.pred(6, 1)

iter = 1000 , loss = 0.5084172106352323 , RMSE train = 0.7956839302328617


2.9383885903706455

In [2]:
print(rs.X.dot(rs.W) + rs.mu)

[[4.16630642 4.01678609 1.80742372 0.22169978 1.71832455 1.43301458
  2.37177554]
 [3.65911002 3.28625493 2.21633095 0.81555901 2.14421932 1.47958092
  2.93838859]
 [3.99819785 3.85489051 1.87307242 0.47721366 1.87803059 1.42249753
  2.46557633]
 [2.44202566 1.70087393 3.05156457 2.3631947  3.20495182 1.53710736
  4.10167038]
 [2.26016789 1.3202394  3.30156148 2.48931507 3.33022426 1.59219535
  4.44389422]]


# Áp dụng lên MovieLens 100k

In [3]:
r_cols = ['user_id', 'movie_id', 'rating', 'unix_timestamp']

ratings_base = pd.read_csv('ml-100k/ub.base', sep='\t', names=r_cols, encoding='latin-1')
ratings_test = pd.read_csv('ml-100k/ub.test', sep='\t', names=r_cols, encoding='latin-1')

rate_train = ratings_base.as_matrix()
rate_test = ratings_test.as_matrix()

# indices start from 0
rate_train[:, :2] -= 1
rate_test[:, :2] -= 1

In [4]:
rs = MF(rate_train, K = 10, lam = .1, print_every = 10, learning_rate = 0.75, max_iter = 100, user_based = 1)
rs.fit()
# evaluate on test data
RMSE = rs.evaluate_RMSE(rate_test)
print('\nUser-based MF, RMSE =', RMSE)

iter = 10 , loss = 5.679844843719727 , RMSE train = 1.2134743841455768
iter = 20 , loss = 2.6477529143475675 , RMSE train = 1.0384134388934132
iter = 30 , loss = 1.3470480619848444 , RMSE train = 1.0295651719954124
iter = 40 , loss = 0.7545397708158952 , RMSE train = 1.0292161951102414
iter = 50 , loss = 0.48299763943907237 , RMSE train = 1.029211200353962
iter = 60 , loss = 0.35847859806678845 , RMSE train = 1.0292134447835877
iter = 70 , loss = 0.30137583436392074 , RMSE train = 1.0292141193926074
iter = 80 , loss = 0.2751892267727495 , RMSE train = 1.029214282190707
iter = 90 , loss = 0.26318039756500267 , RMSE train = 1.029214319869877
iter = 100 , loss = 0.2576733132447252 , RMSE train = 1.0292143284755244

User-based MF, RMSE = 1.060379904586974


In [5]:
rs = MF(rate_train, K = 10, lam = .1, print_every = 10, learning_rate = 0.75, max_iter = 100, user_based = 0)
rs.fit()
# evaluate on test data
RMSE = rs.evaluate_RMSE(rate_test)
print('\nItem-based MF, RMSE =', RMSE)

/home/tran.thi.hai.ha/anaconda3/lib/python3.6/site-packages/numpy/core/fromnumeric.py:3118: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/home/tran.thi.hai.ha/anaconda3/lib/python3.6/site-packages/numpy/core/_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


iter = 10 , loss = 5.620643203767613 , RMSE train = 1.1772712003291592
iter = 20 , loss = 2.615872987176614 , RMSE train = 1.0052506991517447
iter = 30 , loss = 1.323434806439321 , RMSE train = 0.9965314484349979
iter = 40 , loss = 0.7345330559662131 , RMSE train = 0.9961840002854142
iter = 50 , loss = 0.46463772592381514 , RMSE train = 0.9961781132051393
iter = 60 , loss = 0.34087388313396827 , RMSE train = 0.9961800976970254
iter = 70 , loss = 0.28411753499543047 , RMSE train = 0.9961807105150393
iter = 80 , loss = 0.25808982232420097 , RMSE train = 0.9961808591752493
iter = 90 , loss = 0.2461538723402312 , RMSE train = 0.9961808936561729
iter = 100 , loss = 0.24068021436601864 , RMSE train = 0.9961809015425117

Item-based MF, RMSE = 1.04864245343142


In [6]:
rs = MF(rate_train, K = 2, lam = 0, print_every = 10, learning_rate = 1, max_iter = 100, user_based = 0)
rs.fit()
# evaluate on test data
RMSE = rs.evaluate_RMSE(rate_test)
print('\nItem-based MF, RMSE =', RMSE)

/home/tran.thi.hai.ha/anaconda3/lib/python3.6/site-packages/numpy/core/fromnumeric.py:3118: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/home/tran.thi.hai.ha/anaconda3/lib/python3.6/site-packages/numpy/core/_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


iter = 10 , loss = 1.2913465882583544 , RMSE train = 1.466043249352727
iter = 20 , loss = 1.2095435098549165 , RMSE train = 1.4475011896885122
iter = 30 , loss = 1.1375881133815418 , RMSE train = 1.4298835155738134
iter = 40 , loss = 1.0739214709637226 , RMSE train = 1.4131721934200845
iter = 50 , loss = 1.0172849862150752 , RMSE train = 1.3973462848186207
iter = 60 , loss = 0.9666530551989347 , RMSE train = 1.382352273232488
iter = 70 , loss = 0.9211827930500656 , RMSE train = 1.3681271572217677
iter = 80 , loss = 0.8801760380188551 , RMSE train = 1.3546489100844907
iter = 90 , loss = 0.8430503093015931 , RMSE train = 1.3418247054454864
iter = 100 , loss = 0.8093163770379137 , RMSE train = 1.3296742238678245

Item-based MF, RMSE = 1.4036142440339534


In [7]:
RMSE = rs.evaluate_RMSE(rate_train)

In [8]:
print(RMSE)

1.3296742238678245


# MovieLens 1M

In [10]:
r_cols = ['user_id', 'movie_id', 'rating', 'unix_timestamp']

ratings_base = pd.read_csv('mvl/1M.dat', sep='::', names=r_cols, encoding='latin-1')


/home/tran.thi.hai.ha/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  This is separate from the ipykernel package so we can avoid doing imports until


In [12]:
ratings = ratings_base.as_matrix()
ratings[:, :2] -= 1

from sklearn.model_selection import train_test_split

rate_train, rate_test = train_test_split(ratings, test_size=0.33, random_state=42)
print(rate_train.shape, rate_test.shape)

rs = MF(rate_train, K = 2, lam = 0.1, print_every = 2, learning_rate = 2, max_iter = 10, user_based = 0)
rs.fit()
# evaluate on test data
RMSE = rs.evaluate_RMSE(rate_test)
print('\nItem-based MF, RMSE =', RMSE)

(670140, 4) (330069, 4)


/home/tran.thi.hai.ha/anaconda3/lib/python3.6/site-packages/numpy/core/fromnumeric.py:3118: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/home/tran.thi.hai.ha/anaconda3/lib/python3.6/site-packages/numpy/core/_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


iter = 2 , loss = 6.656964501271948 , RMSE train = 1.1083446487108763
iter = 4 , loss = 4.27122303360183 , RMSE train = 0.9999434333940957
iter = 6 , loss = 2.806523062073617 , RMSE train = 0.9780159221830481
iter = 8 , loss = 1.879602795995094 , RMSE train = 0.9742127135282097
iter = 10 , loss = 1.2881612244523561 , RMSE train = 0.9735525655128454

Item-based MF, RMSE = 0.9814140504619255
